In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv('gdrive/My Drive/learning/cloth_prediction/train.csv')



from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import natsort
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from sklearn import model_selection
from keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU

from zipfile import ZipFile
file_name = "/content/gdrive/My Drive/learning/cloth_prediction/train.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

In [ ]:

TRAIN_DIR = 'training/training'

IMG_SIZE=240
train_data_240=[]
labels_240=[]
for img in tqdm(os.listdir(TRAIN_DIR)):
    path = os.path.join(TRAIN_DIR,img)
    img_data = cv2.imread(path)
    img_data = cv2.resize(img_data, (IMG_SIZE,IMG_SIZE))
    train_data_240.append([np.array(img_data)])
    number = int(img[:-4])
    categ_num = int(solution[solution.id==number].category)
    labels_240.append(categ_num)


In [ ]:

x_train,x_test,y_train,y_test=model_selection.train_test_split(train_data_240,labels_240,
                                                               test_size=0.25,random_state=42)

aug=ImageDataGenerator(rotation_range=25,width_shift_range=0.1,height_shift_range=0.1,
                       shear_range=0.2,horizontal_flip=True,fill_mode="nearest")

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="linear",input_shape=(240,240,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(32,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),padding="same",activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),padding="same",activation="relu"))
model.add(BatchNormalization(axis=-1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024,activation="linear"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(6,activation="softmax"))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

#reshape it
# x_train = x_train/255.0
# x_test = x_test/255.0

train=model.fit_generator(aug.flow(x_train,y_train,batch_size=64),validation_data=(x_test,y_test),
                          steps_per_epoch=len(x_train)/64,epochs=150,verbose=1)

In [ ]:
#Let's save this data, so that we don't need to keep calculating it every
#time we want to play with the neural network model:

import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()


#We can always load it in to our current script, or a totally new one by doing:

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
# output all lines

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
df.Pclass.value_counts(normalize=True).plot(kind="bar")


plt.scatter(df.Survived,df.Age, alpha=0.5)


for x in [1,2,3]:
    df.Age[df.Pclass == x].plot(kind="kde")
plt.legend(("1st", "2nd", "3rd"))

In [ ]:
# show all columns
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
np.arange(1, 2, 0.3) 
# >>  [1.  1.3 1.6 1.9]

np.linspace(1, 2, 7)
# >>  [1. 1.16666667 1.33333333 1.5  1.66666667 1.83333333     2.        ]

In [ ]:
lambda_sum = lambda x, y: x + y
reduce(lambda_sum, [1, 2, 3, 4, 5])

In [ ]:
#check NULL values
df.isnull().values.any()

#check NULL values count
df.isnull().sum().sum()

In [ ]:

# LabelEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
temp = df.copy()
temp.iloc[:,1] = label_encoder.fit_transform(df.iloc[:,1])    #1 is column index
temp.head()




In [ ]:

#Feature Scaling
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
standard_scaler = StandardScaler()
normalizer = Normalizer()
min_max_scaler = MinMaxScaler()
print(standard_scaler.fit_transform(X))
print(normalizer.fit_transform(X))
print(min_max_scaler.fit_transform(X))


# One hot encoding
pd.get_dummies(df.iloc[:,1])
temp = pd.concat([temp,onehot], axis = 1)
temp = temp.drop(labels = ["Tag"],axis = 1)


In [ ]:

#Linear Regression with summary

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

feature_cols = ['Reputation',  'Views', 'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8', 'tag_9']

X = temp[feature_cols]
y = temp.Upvotes

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

model1=sm.OLS(y_train,X_train)
result=model1.fit()

print(result.summary())


In [ ]:

#linear regressio with gradient descent

def linear_regression(X, y, m_current=0, b_current=0, epochs=1000, learning_rate=0.0001):
    N = float(len(y))
    for i in range(epochs):  
        y_current = (m_current * X) + b_current
        cost = sum([data**2 for data in (y-y_current)]) / N
        m_gradient = -(2/N) * sum(X * (y - y_current))
        b_gradient = -(2/N) * sum(y - y_current)
        m_current = m_current - (learning_rate * m_gradient)
        b_current = b_current - (learning_rate * b_gradient)
    return m_current, b_current, cost

In [ ]:
### mean square error and r2_score

from sklearn.metrics import mean_squared_error, r2_score

y_predict = slr.predict(X_test) # predict the Y based on the model
mean_squared_error = mean_squared_error(y_test,y_predict) # calculate mean square error
r2_score = r2_score(y_test,y_predict) #calculate r square

print ('mean square error:',mean_squared_error )
print ('r square:',r2_score )

In [ ]:
# plot ROC curve
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')

# calculate AUC
print metrics.roc_auc_score(y_test, y_prob)

In [ ]:
# OPENCV

In [ ]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randrange

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
face_cascade.load('/usr/local/lib/python3.6/dist-packages/cv2/datahaarcascades/haarcascade_frontalface_default.xml')
print(face_cascade.empty())

In [ ]:
img_ = cv2.imread('train/'+os.listdir('train')[4])
##img1 = cv2.cvtColor(img_,cv2.COLOR_BGR2GRAY)
img1 = cv2.cvtColor(img_,cv2.COLOR_BGR2RGB)


## cv2.cvtColor convert the input rgb image into its grayscale form 

img = cv2.imread('train/'+os.listdir('train')[502])
img2 = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img1)
plt.show()
plt.imshow(img2)
plt.show()

In [ ]:
# print the p-values for the model coefficients
lm1.pvalues

In [ ]:
# You see number of unique item for Species with command below:
dataset['Species'].unique()

# grouping
dataset.groupby('Species').count()

#where
dataset.where(dataset ['Species']=='Iris-setosa') #or
dataset[dataset['SepalLengthCm']>7.2]

# Seperating the data into dependent and independent variables
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values


In [ ]:
# split data according to dependent and in dependent variable
X = iris.iloc[:, :-1].values
y = iris.iloc[:, -1].values

In [ ]:
#Feature scalers
# Standard scaler
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(df)

#min-max Scalar
scaler = preprocessing.MinMaxScaler()

# Robust scaler
scaler = preprocessing.RobustScaler()

#normalizer
scaler = preprocessing.Normalizer()

In [ ]:
## Scatter plot in sns

# Modify the graph above by assigning each species an individual color.
sns.FacetGrid(dataset, hue="Species", size=5) \
   .map(plt.scatter, "SepalLengthCm", "SepalWidthCm") \
   .add_legend()
plt.show()

In [ ]:

#### Spelling Corrector


import re, collections

def tokens(text): 
    """
    Get all words from the corpus
    """
    return re.findall('[a-z]+', text.lower()) 

WORDS = tokens(file('big.txt').read())
WORD_COUNTS = collections.Counter(WORDS)

# top 10 words in corpus
print WORD_COUNTS.most_common(10)


def known(words):
    """
    Return the subset of words that are actually 
    in our WORD_COUNTS dictionary.
    """
    return {w for w in words if w in WORD_COUNTS}


def edits0(word): 
    """
    Return all strings that are zero edits away 
    from the input word (i.e., the word itself).
    """
    return {word}



def edits1(word):
    """
    Return all strings that are one edit away 
    from the input word.
    """
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    def splits(word):
        """
        Return a list of all possible (first, rest) pairs 
        that the input word is made of.
        """
        return [(word[:i], word[i:]) 
                for i in range(len(word)+1)]
                
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    """Return all strings that are two edits away 
    from the input word.
    """
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}
    
    
def correct(word):
    """
    Get the best correct spelling for the input word
    """
    # Priority is for edit distance 0, then 1, then 2
    # else defaults to the input word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=WORD_COUNTS.get)


def correct_match(match):
    """
    Spell-correct word in match, 
    and preserve proper upper/lower/title case.
    """
    
    word = match.group()
    def case_of(text):
        """
        Return the case-function appropriate 
        for text: upper, lower, title, or just str.:
            """
        return (str.upper if text.isupper() else
                str.lower if text.islower() else
                str.title if text.istitle() else
                str)
    return case_of(word)(correct(word.lower()))

    
def correct_text_generic(text):
    """
    Correct all the words within a text, 
    returning the corrected text.
    """
    return re.sub('[a-zA-Z]+', correct_match, text)



print correct_text_generic('fianlly')


from pattern.en import suggest

print suggest('fianlly')
print suggest('flaot')

In [ ]:
#Conversion using label encoder

from sklearn.preprocessing import LabelEncoder

clean_df = train_set.copy()

for f in clean_df.columns:
    if clean_df[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_df[f].values))
        clean_df[f] = label.transform(list(clean_df[f].values))
clean_df.head()

In [ ]:
#Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)

In [ ]:
# AUC 

from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(x_train, y_train)

GradientBoostingClassifier(criterion=’friedman_mse’, init=None, learning_rate=0.1, loss=’deviance’, max_depth=3,
max_features=None, max_leaf_nodes=None, min_impurity_split=1e-07, min_samples_leaf=1, min_samples_split=2,
min_weight_fraction_leaf=0.0, n_estimators=100, presort=’auto’, random_state=None, subsample=1.0,
verbose=0, warm_start=False)

y_pred = model.predict(x_test)

from sklearn.metrics import roc_curve, auc

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)

roc_auc = auc(false_positive_rate, true_positive_rate)

roc_auc


> 0.79286211093722836

In [ ]:
NLTK

In [ ]:
#removing text with @

combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*") 
combi.head()

#Removing Punctuations, Numbers, and Special Characters
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
combi.head(10)

#Removing Short Words
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

#Text Normalization
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing


# normalization of tokenised or stemming
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 